<a href="https://colab.research.google.com/github/showpicep/NN/blob/main/%D0%9A%D1%80%D1%83%D1%82%D0%BE%D0%B9_%D1%82%D0%BE%D1%80%D1%87_%D0%B4%D0%BB%D1%8F_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git init
!gh repo clone showpicep/NN

Initialized empty Git repository in /content/.git/
/bin/bash: gh: command not found


In [58]:
import torch
import random
import pandas as pd
import numpy as np

# Устанавливаем фиксированную случайность, для того чтобы сделать код воспроизводимым( значения при повторных запусках проиг не будут обновляться)
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [59]:
# импортнули датасет из скалерна  
import sklearn.datasets
wine = sklearn.datasets.load_wine()
print(wine.data.shape)

(178, 13)


In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    wine.data[:, :13],  # непонятно зачем берем первые две колонки, чел сказал для удобства 
    wine.target, # берем значения классов вин 
    test_size=0.3, # Выделяем 0.3 от всего датасета в тест выборку
    shuffle=True,# Перед делением перемешиваем датасет
    stratify = wine.target# это помогает сделать сбалансированные выборки, то есть чтобы везде были равные пропорции классов
    ) 

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [65]:
class WineNet(torch.nn.Module):
  def __init__(self,n_hidden_neurons):
    super(WineNet,self).__init__()# вот эта строчка непонятная, а все дальше збс

    self.fc1 = torch.nn.Linear(13,n_hidden_neurons)
    self.act1 = torch.nn.Sigmoid()

    # ctrl + '/'
    # self.fc2 = torch.nn.Linear(n_hidden_neurons,n_hidden_neurons)
    # self.act2 = torch.nn.Sigmoid()

    self.fc3 = torch.nn.Linear(n_hidden_neurons,3)

    self.sm = torch.nn.Softmax(dim=1)

  def forward(self,x):
    x = self.fc1(x)
    x = self.act1(x)
    # x = self.fc2(x)
    # x = self.act2(x)
    x = self.fc3(x)
    return x

  def inference(self,x):
    x = self.forward(x)
    x = self.sm(x)
    return x

wine_net = WineNet(13)

In [66]:
loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(wine_net.parameters(), 
                             lr=1.0e-3)

In [67]:
np.random.permutation(5) #получаем пашафленный(перемешанный) набор индексов для массива размерностью 5 

array([0, 2, 1, 3, 4])

In [69]:
batch_size = 20 #Размер "порции"

for epoch in range(10000):
    order = np.random.permutation(len(X_train)) # шафлим 
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = wine_net.forward(x_batch) 
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        test_preds = wine_net.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test).float().mean())

tensor(0.6667)
tensor(0.7222)
tensor(0.6667)
tensor(0.6667)
tensor(0.6667)
tensor(0.8889)
tensor(0.9259)
tensor(0.9259)
tensor(0.9259)
tensor(0.9259)
tensor(0.9630)
tensor(0.9444)
tensor(0.9815)
tensor(0.9444)
tensor(0.9259)
tensor(0.9259)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9630)
tensor(0.9444)
tensor(0.9630)
tensor(0.9259)
tensor(0.9630)
tensor(0.9630)
tensor(0.9630)
tensor(0.9444)
tensor(0.9630)
tensor(0.9444)
tensor(0.9444)
tensor(0.9630)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9630)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9444)
tensor(0.9

In [71]:
wine_net.parameters()

<generator object Module.parameters at 0x7fd30a375550>